# <center>Deep Generative Models</center>
## <center>Seminar 10</center>

<center>12.11.2024</center>

# Plan

1. Gaussian Difussion process
2. Denoising diffusion propbabilistic model
3. Denoising diffusion implicit models

# Noise Conditioned Scored Network
Что помните?

1. Зашумляем с использованием разных $\sigma_1 < \sigma_2 ... < \sigma_T$
2. Учим модель для разных $\sigma$. Модель знает, для какого $t$ она предсказывает шум
3. Семплируем при помощи *Langevin dynamic* - итеративный процесс, когда по чуть-чуть "скатываемся" в $\pi(x)$
4. В конце процесса зашумления получаем $N(0, \sigma_T \cdot I)$ - чтобы можно было семплировать

<center><img src="attachments/Pasted image 20240508023849.png" width=700 /></center>


<center><img src="attachments/Pasted image 20240508023849.png" width=900 /></center>


**Вопрос:** Если мы умеем хорошо предсказывать шум / "расшумлять", то почему бы сразу не расшумлять?

<center><img src="attachments/Pasted image 20240508023849.png" width=900 /></center>


**Вопрос:** Как учить?

**Как учить?**
<center><img src="attachments/Pasted image 20240430191355.png" width=1000 /></center>

**Вопрос:** Как семплировать?

**Как семплировать?**

<center><img src="attachments/Pasted image 20240508023930.png" width=1000 /></center>

**Вопрос:** Какой критерий остановки?

**Примечание**

Чтобы обусловиться на $\sigma_t$ у score-модели мы учим $s(x_t, t)$. Например: 

In [ ]:
class ConditionedResnetBlock(nn.Module):
    def __init__(self, dim: int, num_embeddings: int) -> None:
        super().__init__()
        # you could experiment with this architecture
        self.block = nn.Sequential(
            nn.Conv2d(dim, dim, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(dim, dim, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(dim, dim, kernel_size=1),
        )
        self.dim = dim
        self.embedding = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=dim)

    def forward(self, x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        time_embed = self.embedding(y).view(-1, self.dim, 1, 1)
        return x + self.block(x + time_embed)

## Вспомним теорию
**Как пришли к этой модели?**

1. **Langevin dynamic / Score matching** - не нужно знать $\pi(x)$, нужно знать $\nabla_x \pi(x)$ и можно итеративно семплировать

<center><img src="attachments/Pasted image 20240429131855.png" width=800 /></center>

<center><img src="attachments/Pasted image 20240429131855.png" width=800 /></center>

**Вопрос:** Не знаем $\pi(x)$, умеем семплировать. Что делать? 

2. **Denosing score matching** - зашумим предсказания и будем предсказывать шум, НО $\nabla_{x_{\sigma}}q(x_{\sigma}|x)$ можно будет посчитать!

<center><img src="attachments/Pasted image 20240508024107.png" width=1000 /></center>

**Вопрос:** В чем обман?

<center><img src="attachments/Pasted image 20240508024107.png" width=1000 /></center>

1. Ключевая идея теоремы - разложить матожидание для зашумленных семплов $\mathbf{E}_{q(x_{\sigma})} = \mathbf{E}_{\pi(x)}\mathbf{E}_{q(x_{\sigma}|x)}$
2. **В чем обман?** "локальная" score функция

## Итого

1. **Langevin dynamic / Score matching** - не знаем $\pi(x)$, но знаем $\nabla_x \pi(x)$ $\Rightarrow$ можно семплировать из $\pi(x)$
2. **Denosing score matching** - обучаем модель денойзить, получим "локальные" score функции 
3. **Noice Conditioned Score Network** - учим $s(t, x_t)$ для набора $\sigma_1 < \sigma_2 ... < \sigma_T$
	1. $q(x_1) \sim \pi(x), q(x_T) \sim N(0, \sigma_T \cdot I)$
	2. Семплируем при помощи *Langevin dynamic*

## Gaussian Diffusion process

Что помните?

1. **Forward gaussian diffusion process** - по чуть-чуть "впрыскивают" шум, картинка превращается в $N(0, I)$
2. **Reverse gaussian diffusion process** - восстанавливаем картинку из $N(0, I)$, учим модель предсказывать шум и по чуть-чуть денойзим
3. На практике - учим модель денойзить зашумленную картинку

## Вспоминаем математику

Идея - можем постепенно зашумить картинку так, чтобы прийти в $N(0, I)$

<center><img src="attachments/Pasted image 20240508024010.png" width=1000 /></center>

<center><img src="attachments/Pasted image 20240508024215.png" width=1000 /></center>

<center><img src="attachments/Pasted image 20240508024215.png" width=1000 /></center>

**Примечания:**
1. $q(x_{\infty}|x_{0}) = N(0, I)$ - домнажая на число 0 < $\sqrt{1 - \beta}$ < 1 размываем исходную картинку
2. $p_{\infty}(x) = N(0, I)$ - мат. ожидание размыли в 0, намешали кучу нормального шума - получили нормальный шум
3. Марковость $q(x_{t}|x_{t-1})$ - нужно будет в выводе того, как делать обратный процесс
4. Процесс определяется $\beta$-policy и $T$ 

<center><img src="attachments/Pasted image 20240508024215.png" width=1000 /></center>

**Вопрос:** Можем ли мы явно обратить процесс?

1. $q(x_{t-1}|x_{t}) = \dfrac{q(x_t|x_{t-1})q(x_{t-1})}{q(x_t)}$
2. $q(x_t) = \int p(x_t|x_0) \pi(x_0) dx_0$ - мы такое считать не умеем

<center><img src="attachments/Pasted image 20240508024215.png" width=1000 /></center>

**Вопрос:** Можем ли сразу зашумлять данные так, чтобы все явно обращалось? 

<center><img src="attachments/Pasted image 20240508024215.png" width=1000 /></center>

**Вопрос:** Что делаем, когда не знаем как посчитать $q(x_{t-1}|x_t)$?

Учим модель $p_{\theta, t}(x_{t-1}|x_t)$

<center><img src="attachments/Pasted image 20240508024215.png" width=1000 /></center>

**Вопрос:** Что мы знаем про $q(x_{t-1}|x_{t})$?

1. Учим модель не ответ - давайте сразу обучим $x \sim \pi(x)$ предсказывать
2. Когда мы учили модель предсказывать распределения:
    - модель предсказывала **параметры**, которые задают распределение
    - это были либо нормальные, либо категориальные распределения

## Conditioned reverse distribution

Давайте упростим $q(x_{t-1}|x_{t})$. Как это делали на лекции?

**Подсказка:** Почему у диффузионных моделей долгое семплирование?

**Feller theorem 1949** - при мальеньких шагах диффузии обратное преобразование тоже будет гауссовским

<center><img src="attachments/Pasted image 20240507204248.png" width=1000 /></center>

**Вопрос:** Мы по прежнему не можем явно вычислить $q(x_{t-1}|x_t)$. Что еще можно упростить, чтобы явно выписать $\mu, \sigma$?

$q(x_{t-1}|x_{t})$ мы посчитать не можем, НО $q(x_{t-1}|x_t, x_0)$ можем!

<center><img src="attachments/Pasted image 20240501143147.png" width=1000 /></center>

**Вопрос:** В чем обман?

$q(x_{t-1}|x_{t})$ мы посчитать не можем, НО $q(x_{t-1}|x_t, x_0)$ можем!

<center><img src="attachments/Pasted image 20240501143147.png" width=1000 /></center>

**Вопрос:** В чем обман?

1. С точки зрения математики ни в чем, все формулы корректны
2. *Интуиция* ака *махание руками* - мы из нормального шума можем получить все, что угодно, поэтому непонятно "куда идти". Если же знаем конечное $x_0$ куда хотим расшумить, становится чуть проще

Теперь можно параметризовать $q(x_{t-1}|x_t, x_0)$ и учить модели.

<center><img src="attachments/Pasted image 20240430204919.png" width=1000 /></center>

**Вопрос:** С какой loss function?

<center><img src="attachments/Pasted image 20240507205459.png" width=1200 /></center>

## VAE recap

**А откуда вообще берется ELBO?**

5 семинар про VAE 

> The logic of the process:
> 1. Introduce Latent Variable Model (**LVM**).
> 2. Derive the Evidence Lower Bound (**ELBO**), notice that $argmax_q L = argmin_q KL[q, p(z|x, \theta)] = p(z|x, \theta)$.
> 3. **EM algorithm** (tighten the lower bound on ELBO, thereby tightening the likelihood).
> 4. Introduce a **parametric** $q(z|x, \phi)$, transitioning to amortized variational inference (gradient optimization)


1. Вводим LVM

<center><img src="attachments/Pasted image 20240507213823.png" width=1200 /></center>

1. Вводим LVM

<center><img src="attachments/Pasted image 20240507213850.png" width=1200 /></center>

2. Выписываем ELBO

<center><img src="attachments/Pasted image 20240507214229.png" width=1200 /></center>

В конце вывода останется

<center><img src="attachments/Pasted image 20240501155653.png" width=1200 /></center>

Фактически "обучение модели" значит - обучить $p(x_{t-1}|x_{t})$ приближать $q(x_{t-1}|x_{t}, x_0)$

<center><img src="attachments/Pasted image 20240501160106.png" width=1000 /></center>

**Вопрос:** Как почситать $L_t$?

Это $KL$ между двумя нормальными распределениями, при этом $q(x_{t-1}|x_t, x_0)$ мы явно считать умеем

## Несколько трюков / упрощений

1. **Будем явно считать** $\sigma_{\theta,t}$. У распределения которое мы учим 2 параметра - $\mu, \sigma$. Есть теоретические оценики на дисперсию, можно посчитать ее аналитически так же, как мы считаем для вычислимого $q(x_{t-1}|x_t, x_0)$. Это сильно упростит вид KL и loss функция будет зависеть только от $\mu$

<center><img src="attachments/Pasted image 20240501203805.png" width=1200 /></center>

2. **Репараметризация через $x_t$ и $\epsilon$**. Мат. ожидание $q(x_{t-1}|x_t, x_0)$ считается через $x_t$ и $x_0$, при этом $x_t$ считается из $x_0$ и $\epsilon$ $\Rightarrow$ выразим мат. ожидание через $x_t$ и нормальный шум $\epsilon$

<center><img src="attachments/Pasted image 20240501204235.png" width=1200 /></center>

3. **Вынесем константы.**  Cократим $x_t$, вынесем все константы за скобки и оставить только шум

<center><img src="attachments/Pasted image 20240501204436.png" width=1200 /></center>

Получили loss function - "ну давайте шум предсказывать"

<center><img src="attachments/Pasted image 20240501205916.png" width=1200 /></center>

## Итого

С точки зрения математики:
1. **Forward gaussian diffusion process** - *марково* зашумляем картинку, постепенно размывая исходное изображение  в $N(0, I)$ ("добавка", которую подмешиваем $q(x_t|x_{t-1}) \rightarrow N(0, I)$)

2. **Feller theorem**. Нужно научиться считать обратное преобразование. Если мы будем портить картинку "по чуть-чуть", то **и обратное преобразование будет гауссово** 


3. Чтобы обучить модель "делать обратное преобразование" - вводим **LVM**, расписываем **ELBO**

4. ELBO - получаем лосс из 3 слагаемых:
	1. **Reconstruction locc** с маленьким зашумлением, тоже денойзер
	2. $\mathbf{KL}(p(x_T|x_0)|p(x_T))$ $\approx 0$ по построению процесса прямой диффузии - все переходит в $N(0, I)$
	3. $\mathbf{KL}(q(x_{t-1}|x_t, x_0)| p(x_{t-1}|x_t, \theta))$ между **выученным обратным преобразованием** с и **обратным, которое мы можем посчитать явно** (из-за обусловленности на $x_0$, которая возникла в выводе ELBO)


На практике: учим модель предсказывать каким шумом зашумили картинку

## Denoising diffusion propbabilistic model

Соберем все вместе

<center><img src="attachments/Pasted image 20240501211003.png" width=1200 /></center>

Кто может описать, как происходит **процесс обучения**?

**Замечания:**
1. Семплируем $t, \epsilon$ , считаем $x_t$ через $x_0$ и $\epsilon$
2. Модель предсказывает $\epsilon$ - шум, который добавили к $x_0$
3. Модель обусловлена на $t$ - мы в нее тоже каким-то образом сразу подаем знание о том, для какого $t$ предсказываем шум
4. Policy $\beta$ фактически задает и прямой и обратный диффузионный процесс


Кто может описать, как происходит **процесс семплирования**?

1. Cемплируем $x_T = \epsilon$ из $N(0, I)$
2. Cчитаем $x_{t-1}$ зная $x_t$
	1. Обученым денойзером предсказываем шум $\epsilon_{\theta}(x_t, t)$
	2. Считаем мат. ожидание $\mu_{\theta}(x_t, t, \epsilon_\theta)$
	3. К $\mu_{\theta}(x_t, t, \epsilon_\theta)$ добавляем шум $\epsilon \sim N(0, I)$ умноженный на дисперсию


<center><img src="attachments/Pasted image 20240501211039.png" width=1200 /></center>

**Вопрос:** А что будет, если не добавлять шум $\epsilon$ в семплировании?

**Вопрос:** Какие проблемы есть в текущем сетапе?

1. Guidance
2. Slow sampling

## Sampling

### Problem 1: Faster sampling with DDPM (spaced diffusion) (1pt)

Sampling from DDPM is very slow. In the practical part of HW you will see that it took about 8 seconds to generate a batch of images with our diffusion model (even using a gpu).
While, as you might remember, it took less than a second using other generative models (VAE/GAN/NF).
This drawback can't be solved generally with using more gpus, since it requires iterative sampling.
There are several techniques to alleviate this drawback. In this task We are going to investigate one of them.

Assume we have already trained a model $p(\mathbf{x}_{t - 1} | \mathbf{x}_t, \boldsymbol{\theta})$ to "reverse" a Markov chain of length $T$.

Let try to build inference process using subsequence of timesteps
$\{S_0 = 0, S_1, \ldots, S_{T'-1}, S_{T'} = T\}$, where $T' < T$.

Using this subsequence we have to do $T' (< T)$ inference steps instead of $T$. It could dramatically reduce inference time.

Diffusion models inference are essentially defined by
- schedule of variances $\{\beta_1, \ldots, \beta_T\}$
- reverse process:
$$
p(\mathbf{x}_{S_{t - 1}} | \mathbf{x}_{S_t}, \boldsymbol{\theta}) = \mathcal{N} \bigl(\mathbf{x}_{S_{t - 1}} | \boldsymbol{\mu}_{\boldsymbol{\theta}}(\mathbf{x}_{S_t}, S_t), \tilde{\beta}_{S_t}\bigr)
$$

Therefore, all you have to find is the variances for the new Markov chain: $\{\tilde{\beta}_{S_1}, \ldots, \tilde{\beta}_{S_{T'}}\}$.

**Task:** find the expression for $\tilde{\beta}_{S_t}$ (it should depend on $\alpha$).